# Feature Analysis & Visualization

This notebook visualizes the advanced features generated by the feature engineering pipeline.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

In [ ]:
# Load Data
data_path = Path('../data/BTCUSDT_2021_2023_1m_features.csv')

if not data_path.exists():
    print(f"Data file not found at {data_path}. Please run feature engineering first.")
else:
    df = pd.read_csv(data_path)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.set_index('timestamp').sort_index()
    print(f"Loaded {len(df)} rows.")
    print(df.columns.tolist())

## 1. Trend Indicators

In [ ]:
# Plot a sample period (e.g., last 10000 points or a specific volatile period)
sample = df.iloc[-5000:]

plt.figure(figsize=(15, 8))
plt.plot(sample.index, sample['Close'], label='Close', alpha=0.5, color='black')
if 'sma_30d' in sample.columns: plt.plot(sample.index, sample['sma_30d'], label='SMA 30d')
if 'ema_21d' in sample.columns: plt.plot(sample.index, sample['ema_21d'], label='EMA 21d')
plt.title('Price vs Trend Indicators')
plt.legend()
plt.show()

## 2. Momentum & RSI Regimes

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10), sharex=True)

ax1.plot(sample.index, sample['Close'], label='Close')
ax1.set_title('Price')

if 'rsi_14d' in sample.columns:
    ax2.plot(sample.index, sample['rsi_14d'], label='RSI 14d', color='purple')
    ax2.axhline(70, linestyle='--', color='red', alpha=0.5)
    ax2.axhline(30, linestyle='--', color='green', alpha=0.5)
    
    # Plot dynamic bands if available
    if 'rsi_14d_q70' in sample.columns:
        ax2.plot(sample.index, sample['rsi_14d_q70'], linestyle=':', color='red', label='Dynamic 70th')
        ax2.plot(sample.index, sample['rsi_14d_q30'], linestyle=':', color='green', label='Dynamic 30th')
        
    ax2.set_title('RSI & Dynamic Regimes')
    ax2.legend()

plt.show()

## 3. Volatility Regimes

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10), sharex=True)

ax1.plot(sample.index, sample['Close'], label='Close')
if 'bb_upper_20d' in sample.columns:
    ax1.plot(sample.index, sample['bb_upper_20d'], 'r--', alpha=0.3)
    ax1.plot(sample.index, sample['bb_lower_20d'], 'g--', alpha=0.3)
ax1.set_title('Bollinger Bands')

if 'volatility_ratio_7d' in sample.columns:
    ax2.plot(sample.index, sample['volatility_ratio_7d'], label='Vol Ratio (vs 28d avg)', color='orange')
    ax2.axhline(1.0, linestyle='--', color='gray')
    ax2.set_title('Volatility Ratio')
    ax2.legend()

plt.show()

## 4. Fractal Dimension

In [ ]:
if 'fd_7d' in sample.columns:
    fig, ax1 = plt.subplots(figsize=(15, 6))

    color = 'tab:blue'
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Close', color=color)
    ax1.plot(sample.index, sample['Close'], color=color, alpha=0.5)
    ax1.tick_params(axis='y', labelcolor=color)

    ax2 = ax1.twinx()
    color = 'tab:red'
    ax2.set_ylabel('Fractal Dimension (7d)', color=color)
    ax2.plot(sample.index, sample['fd_7d'], color=color, alpha=0.8)
    ax2.tick_params(axis='y', labelcolor=color)

    plt.title('Price vs Fractal Dimension')
    plt.show()

## 5. Feature Correlations

In [ ]:
# Select numeric columns
cols = [c for c in sample.columns if sample[c].dtype in [np.float64, np.int64]]
# Filter out some raw columns to keep matrix readable
cols = [c for c in cols if 'Open' not in c and 'High' not in c and 'Low' not in c and 'timestamp' not in c]

corr = sample[cols].corr()

plt.figure(figsize=(20, 16))
sns.heatmap(corr, annot=False, cmap='coolwarm', center=0)
plt.title('Feature Correlation Matrix')
plt.show()